In [74]:
from fpdf import FPDF
class PDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, 'MCQ Quiz', 0, 1, 'C')

    def chapter_title(self, num, label):
        self.set_font('Arial', '', 12)
        self.cell(0, 10, 'Question %d: %s' % (num, label), 0, 1, 'L')
        self.ln(5)

    def chapter_body(self, body):
        self.set_font('Arial', '', 12)
        self.multi_cell(0, 10, body)
        self.ln()

    def add_question(self, num, question, options):
        self.chapter_title(num, question)
        for key, option in options.items():
            self.chapter_body(f"{key}. {option}")
        self.ln()

    def add_answers_section(self, answers):
        self.add_page()
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, 'Answers', 0, 1, 'C')
        self.ln(10)
        self.set_font('Arial', '', 12)
        for num, answer in answers.items():
            self.cell(0, 10, f"Question {num}: {answer}", 0, 1, 'L')

In [28]:
import os
import json
import pandas as pd
import traceback


In [29]:
import google.generativeai as genai

In [30]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [37]:
from dotenv import load_dotenv

load_dotenv()

True

In [38]:
import os
KEY=os.getenv("GEMINI_API_KEY")

In [39]:
llm = ChatGoogleGenerativeAI(model="gemini-pro",google_api_key=KEY,temperature=0.5)


In [41]:
llm

ChatGoogleGenerativeAI(model='models/gemini-pro', google_api_key=SecretStr('**********'), temperature=0.5, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000001D4E011E020>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x000001D4E0157970>, default_metadata=())

In [42]:

from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
# from langchain.callbacks import get_openai_callback
import PyPDF2

In [43]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [44]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [45]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [46]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

c:\Users\SHIVAM\mcqgen\env\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [47]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [48]:
quiz_evaluation_prompt=PromptTemplate(
    input_variables=["subject","quiz"],
    template=TEMPLATE2
)

In [49]:
review_chain=LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [50]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain,review_chain],input_variables=["text","number","subject","tone","response_json"],
                                         output_variables=["quiz", "review"], verbose=True,)

In [51]:
file_path=r"C:\Users\SHIVAM\mcqgen\data.txt"

In [52]:
with open(file_path,'r') as file:
    TEXT = file.read()

In [53]:
print(TEXT)

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive systems contributed 

In [54]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [55]:
NUMBER=5 
SUBJECT="machine learning"
TONE="simple"

In [56]:
response=generate_evaluate_chain(
            {
                "text": TEXT,
                "number": NUMBER,
                "subject":SUBJECT,
                "tone": TONE,
                "response_json": json.dumps(RESPONSE_JSON)
            }
)


c:\Users\SHIVAM\mcqgen\env\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used b

In [67]:
quiz=response.get("quiz")

In [68]:
print(quiz)

### RESPONSE_JSON
{"1": {"mcq": "Who is credited with coining the term 'machine learning'?", "options": {"a": "Donald Hebb", "b": "Arthur Samuel", "c": "Walter Pitts", "d": "Raytheon Company"}, "correct": "b"}, "2": {"mcq": "What was the purpose of the 'goof' button on the Cybertron learning machine?", "options": {"a": "To reset the machine's memory", "b": "To cause the machine to reevaluate incorrect decisions", "c": "To provide feedback to the human operator", "d": "To stop the machine's learning process"}, "correct": "b"}, "3": {"mcq": "What is a key difference between the early definition of machine learning and the modern definition?", "options": {"a": "The early definition focused on cognitive terms, while the modern definition is more operational.", "b": "The modern definition includes prediction as a goal, while the early definition did not.", "c": "The early definition was specific to pattern recognition, while the modern definition is more general.", "d": "The modern definiti

In [71]:
if '### RESPONSE_JSON\n' in quiz:
    quiz = quiz.split('### RESPONSE_JSON\n')[1]
    quiz = json.loads(quiz)
else:
    quiz = json.loads(quiz)


In [73]:
quiz

{'1': {'mcq': "Who is credited with coining the term 'machine learning'?",
  'options': {'a': 'Donald Hebb',
   'b': 'Arthur Samuel',
   'c': 'Walter Pitts',
   'd': 'Raytheon Company'},
  'correct': 'b'},
 '2': {'mcq': "What was the purpose of the 'goof' button on the Cybertron learning machine?",
  'options': {'a': "To reset the machine's memory",
   'b': 'To cause the machine to reevaluate incorrect decisions',
   'c': 'To provide feedback to the human operator',
   'd': "To stop the machine's learning process"},
  'correct': 'b'},
 '3': {'mcq': 'What is a key difference between the early definition of machine learning and the modern definition?',
  'options': {'a': 'The early definition focused on cognitive terms, while the modern definition is more operational.',
   'b': 'The modern definition includes prediction as a goal, while the early definition did not.',
   'c': 'The early definition was specific to pattern recognition, while the modern definition is more general.',
   'd':

In [81]:
import streamlit as st
import json
import base64

# Initialize the PDF
pdf = PDF()
pdf.add_page()
pdf.set_title(SUBJECT + " Quiz")
answers = {}

# Add questions and collect answers
for key, value in quiz.items():
    question_num = int(key)
    pdf.add_question(question_num, value["mcq"], value["options"])
    answers[question_num] = value["correct"]

# Add answers section to the PDF
pdf.add_answers_section(answers)

# Define the file path for the PDF
pdf_file_path = SUBJECT + "_mcq.pdf"
pdf.output(pdf_file_path)

# Create a download button in Streamlit
with open(pdf_file_path, "rb") as pdf_file:
    st.download_button(
        label="Download " + SUBJECT + " Quiz PDF",
        data=pdf_file,
        file_name=SUBJECT + "_quiz.pdf",
        mime="application/pdf",
    )

# # Display the PDF in Streamlit
pdf_display = f'<iframe src="data:application/pdf;base64,{base64.b64encode(open(pdf_file_path, "rb").read()).decode()}" width="700" height="1000" type="application/pdf"></iframe>'
st.markdown(pdf_display, unsafe_allow_html=True)


DeltaGenerator()